In [ ]:
!pip install asrpy -q
!pip install mne

In [ ]:
import sklearn
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from asrpy import ASR
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import pandas as pd
import csv
import seaborn as sns
from scipy.integrate import simpson
from scipy import signal
import scipy.stats
import mne
from mne.preprocessing import ICA
from google.colab import drive
import pickle

drive.mount('/content/drive')

## Data visualization and cleaning

In [ ]:
# name of folder all raw data is in (no '/' at the end)
folder_name = f"/content/drive/Shared drivers/NeurotechX Shared Drive/Alzheimer's Dataset"


#what subject number to look at (integer)
subject = 5
file_path = f"/content/drive/Shared drives/NeurotechX Shared Drive/Alzheimer's Dataset/derivatives/sub-{subject:03}/eeg/sub-{subject:03}_task-eyesclosed_eeg.set"


In [ ]:
def visualize_patient(file_path):

    #Import data and preprocess it
    # 1) Filter to 0.5-50 Hz
    # 2) Resample to 120 Hz
    # 3) Re-reference EEG to average value
    raw_data = mne.io.read_raw_eeglab(file_path)
    raw_data.filter(l_freq=0.5, h_freq=50.0)
    raw_data.resample(sfreq=120)
    raw_data.set_eeg_reference(ref_channels='average')

    # Apply Artifact Subspace Reconstruction -> For artifact removal
    asr = ASR(sfreq=raw_data.info["sfreq"], cutoff=15)
    asr.fit(raw_data)
    raw_data = asr.transform(raw_data)

    #visualize raw EEG recordings
    raw_data.plot(duration=4, n_channels=19)

    #Separate signals into independent components using ICA and visualize
    ica = ICA(n_components=19)
    ica.fit(raw_data)
    ica.plot_components()
    #fig.savefig('components.png')

    ica.plot_sources(raw_data)

# Visualizing one patient's data
print('\n\nVisualizing data for patient', subject, '\n\n')
visualize_patient(file_path)

#### Epoch signal

In [ ]:
def epoch_signal(signal_data, epoch_length, overlap, sampling_rate):
    # Calculate the number of samples per epoch
    samples_per_epoch = int(epoch_length * sampling_rate)

    # Calculate the step size for the overlap
    step_size = int(samples_per_epoch * (1 - overlap))

    # Create an array to store the epochs
    epochs = []

    # Iterate over the signal_data with the given step size
    for i in range(0, signal_data.shape[1] - samples_per_epoch + 1, step_size):
        # Extract the current epoch
        epoch = signal_data[:, i:i+samples_per_epoch]

        # Append the epoch to the list of epochs
        epochs.append(epoch)

    # Convert the list of epochs to a numpy array
    epochs = np.array(epochs)

    return epochs

Calculate relative band power

In [ ]:
def get_bandpowers(subj_epochs, sf=120, win_len=480):
    freqs, psd = signal.welch(subj_epochs, sf, nperseg=win_len, axis=2)

    freq_res = freqs[1] - freqs[0]
    total_power = simpson(psd, freqs, axis=2)

    def get_rel_power(low, high, total_power, freqs, psd):
        idx_select = np.logical_and(freqs >= low, freqs <= high)
        band_power = simpson(psd[:, :, idx_select], freqs[idx_select], axis=2)
        rel_power = band_power / total_power
        return rel_power

    #delta band 0.5-4 Hz
    delta_power = get_rel_power(0.5, 4, total_power, freqs, psd)
    #theta band 4-8 Hz
    theta_power = get_rel_power(4, 8, total_power, freqs, psd)
    #alpha band 8-13 Hz
    alpha_power = get_rel_power(8, 13, total_power, freqs, psd)
    #beta band 13-25 Hz
    beta_power = get_rel_power(13, 25, total_power, freqs, psd)
    #gamma band 25-45 Hz
    gamma_power = get_rel_power(25, 45, total_power, freqs, psd)

    return delta_power, theta_power, alpha_power, beta_power, gamma_power


### Processing pipeline

In [ ]:
# Pulls participant group type from participants.tsv file
# A: Alzheimer group; C: Healthy (Control) group; F: Frontotemporal Dementia group
subj_types = []
participants_path = f"/content/drive/Shared drives/NeurotechX Shared Drive/Alzheimer's Dataset/participants.tsv"
with open(participants_path) as file:
  for line in file:
    l = line.split('\t')
    subj_types.append(l[3])

In [ ]:
def get_subject_features(file_path):
    #get raw data and pre-process it
    raw_data = mne.io.read_raw_eeglab(file_path, verbose=False)
    raw_data.filter(l_freq=0.5, h_freq=50.0, verbose=False)
    raw_data.resample(sfreq=120)
    raw_data.set_eeg_reference(ref_channels='average')

    sf = raw_data.info['sfreq']
    win_len = 4 * sf #step size of 0.25

    # Apply the ASR
    asr = ASR(sfreq=sf, cutoff=15)
    asr.fit(raw_data)
    raw_data = asr.transform(raw_data)

    #Try to remove artifacts with ICA
    ica = ICA(n_components=19)
    ica.fit(raw_data, verbose=False)

    #get ica data
    ica_data = ica.apply(raw_data, verbose=False).get_data()
    epoch_length = 4  # 4 seconds per epoch
    overlap = 0.5  # 50% overlap
    sampling_rate = 120  # Signal is sampled at 120 Hz

    epochs = epoch_signal(ica_data, epoch_length, overlap, sampling_rate)
    patient_epoch = epochs.shape[0]

    #get time series features
    #get average
    averages = np.mean(epochs, axis=2)
    #get std
    stds = np.std(epochs, axis=2)
    #get IQR
    iqrs = scipy.stats.iqr(epochs, axis=2, rng=(25,75))


    #get frequency features: relative spectral power density for EEG Rhythms
    delta_power, theta_power, alpha_power, beta_power, gamma_power = get_bandpowers(epochs, sf)

    #put everything together in a dataframe: columns are features, rows are 4 second epochs that they are for
    features = ['average', 'std', 'iqr', 'delta', 'theta', 'alpha', 'beta', 'gamma',]
    col_names = [f'{x}_{y}' for y in features for x in raw_data.ch_names]
    subject_df = pd.DataFrame(np.concatenate([averages, stds, iqrs, delta_power, theta_power, alpha_power, beta_power, gamma_power], axis=1), columns=col_names)
    return subject_df, patient_epoch

In [ ]:
def extract_all_features(folder_name):
    # Iterates through all patients to extract features and organize them into a single dataframe
    patient_features_list = []
    patient_epoch_length = []
    for subject in range(1, 89):
        print("Starting for patient ", subject)
        file_path = folder_name + f"/sub-{subject:03}/eeg/sub-{subject:03}_task-eyesclosed_eeg.set"
        all_features, patient_epoch = get_subject_features(file_path)
        print("Completed features extraction for patient ", subject)
        print('\n\n\n')
        #print(all_features)
        patient_features_list.append(all_features)
        patient_epoch_length.append(patient_epoch)

    all_patient_features = pd.concat(patient_features_list).reset_index(drop=True)

    return all_patient_features, patient_epoch_length


In [ ]:
# Extracting all patient features, preprocessing and generating data

all_patient_features, patient_epoch_length = extract_all_features(folder_name)

In [ ]:
# Exports data and classification labels to shared drive
# This has been commented out because the data's already been exported!
# Uncomment the code below if you would like to export the data.
# Be sure to change the filename.


# all_patient_features.to_csv('/content/drive/Shared drives/NeurotechX Shared Drive/all_patient_features_w_artifact_removal_raw.csv', index=False)

# with open('/content/drive/Shared drives/NeurotechX Shared Drive/all_subj_types_w_artifact_removal_raw.csv', 'w', encoding="ISO-8859-1", newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(all_subj_types)

## Prepare Classifier Inputs

In [ ]:
def split_data(all_patient_features, patient_epoch_length, subj_types, test_fraction):

    # epochs_by_patient is an array of integers, where the element at the i-th index is the subsection of the
    # all_patient_features dataframe consisting of all the epochs corresponding to the patient
    epochs_by_patient = []
    slice_start = 0
    for subject in range(len(patient_epoch_length)):
        epochs_by_patient.append(all_patient_features[slice_start : slice_start + patient_epoch_length[subject]].to_numpy())
        slice_start += patient_epoch_length[subject]

    # Renaming data_values to X and sample_class to y
    # X = data_values
    X = epochs_by_patient # list of 88 elements, i-th element contains the dataframe subsection of features for all epochs corresponding to the i-th subject
    # y = sample_class
    y = subj_types # list of 88 elements, i-th element contains classification group of the i-th subject ("A", "F", "C")


    # Splitting data to train-test using stratified split (by classification group)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_fraction, stratify=y, random_state=1234)

    # Separate epochs from their subject-specific arrays to form combined dataframes
    # (no longer need to know which subject each epoch comes from)
    flat_X_train = []
    flat_X_test = []
    flat_y_train = []
    flat_y_test = []
    for i in range(len(X_train)):
        flat_X_train.extend(X_train[i])
    for i in range(len(X_test)):
        flat_X_test.extend(X_test[i])
    for i in range(len(y_train)):
        flat_y_train.extend([y_train[i]] * len(X_train[i]))
    for i in range(len(y_test)):
        flat_y_test.extend([y_test[i]] * len(X_test[i]))

    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(flat_X_train)
    scaled_X_test = scaler.transform(flat_X_test)

    return scaled_X_train, scaled_X_test, flat_y_train, flat_y_test


## PCA

In [ ]:
from sklearn.decomposition import PCA

def get_pca_data(exp_variance, X_train, X_test, y_train, y_test, plot=False):
    pca = PCA(n_components=exp_variance)
    X_train_pca = pca.fit_transform(X_train)
    #print(X_train_pca.shape)
    X_test_pca = pca.transform(X_test)
    exp_var_pca = pca.explained_variance_ratio_
    if plot:
        cum_sum_eigenvalues = np.cumsum(exp_var_pca)
        #
        # Create the visualization plot
        #
        plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
        plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
        plt.ylabel('Explained variance ratio')
        plt.xlabel('Principal component index')
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

    return X_train, X_test

# sample usage
# X_train_pca, X_test_pca = get_pca_data(0.9, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)

## PLSDA

In [ ]:
from sklearn.cross_decomposition import PLSRegression

def get_plsda_data(num_components, X_train, X_test, y_train, y_test, plot=False):
    plsda = PLSRegression(n_components=num_components)

    # Required for plsda, convert class labels into numerical equivalents
    flat_y_train_num = []
    for label in y_train:
      if label == 'A':
        flat_y_train_num.append(0)
      elif label == 'F':
        flat_y_train_num.append(1)
      elif label == 'C':
        flat_y_train_num.append(2)


    X_train_plsda, y_train_plsda = plsda.fit_transform(X_train, flat_y_train_num)

    X_test_plsda = plsda.transform(X_test)

    return X_train_plsda, X_test_plsda

# sample usage
# X_train_plsda, X_test_plsda = get_plsda_data(100, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)

##Random Decision Forest

In [ ]:
scaled_X_train, scaled_X_test, flat_y_train, flat_y_test = split_data(all_patient_features, patient_epoch_length, subj_types, 0.3)

In [ ]:
def run_rf(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test, dim_reduce=None, n_components=0):

    X_train = scaled_X_train
    X_test = scaled_X_test

    # Check if dimensional reduction
    if dim_reduce == 'pca':
        X_train, X_test = get_pca_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)
    elif dim_reduce == 'plsda':
        X_train, X_test = get_plsda_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)


    # n_estimators is the number of decision trees
    # max_features is the number of features each tree uses in decision-making
    clf = RandomForestClassifier(n_estimators=100, max_features='sqrt')
    clf.fit(X_train, flat_y_train)

    # Applies model on test data
    y_pred = clf.predict(X_test) # array of predictions of classifier after training

    # using metrics module for accuracy calculation. Compares predicted vs. actual
    # class labels in test set to calculate accuracy
    accuracy = metrics.accuracy_score(flat_y_test, y_pred)

    return y_pred, accuracy

## SVM

In [ ]:
# Trains model
def run_svm(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test, dim_reduce=None, n_components=0):

    X_train = scaled_X_train
    X_test = scaled_X_test

    if dim_reduce == 'pca':
        X_train, X_test = get_pca_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)
    elif dim_reduce == 'plsda':
        X_train, X_test = get_plsda_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)


    model = svm.SVC(C=1,
                    kernel='poly', #linear, poly, rbf, sigmoid, precomputed
                    )
    model.fit(X_train, flat_y_train)

    # Applies model on test data
    y_pred = model.predict(X_test) # array of predictions of classifier after training

    # using metrics module for accuracy calculation. Compares predicted vs. actual
    # class labels in test set to calculate accuracy
    accuracy = metrics.accuracy_score(flat_y_test, y_pred)

    return y_pred, accuracy

## KNN

In [ ]:
def run_knn(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test, dim_reduce=None, n_components=0):

    X_train = scaled_X_train
    X_test = scaled_X_test

    if dim_reduce == 'pca':
        X_train, X_test = get_pca_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)
    elif dim_reduce == 'plsda':
        X_train, X_test = get_plsda_data(n_components, scaled_X_train, scaled_X_test, flat_y_train, flat_y_test)

    neigh = KNeighborsClassifier(n_neighbors=3)

    # Trains model
    neigh.fit(X_train, flat_y_train)

    # Applies model on test data
    y_pred = neigh.predict(X_test) # array of predictions of classifier after training

    # using metrics module for accuracy calculation. Compares predicted vs. actual
    # class labels in test set to calculate accuracy
    accuracy = metrics.accuracy_score(flat_y_test, y_pred)

    return y_pred, accuracy

## Multi-run all classifiers

In [ ]:
rf_res = []
knn_res = []
svm_res = []
rf_cm = []
knn_cm = []
svm_cm = []
y_test_res = []
rf_acc = []
knn_acc = []
svm_acc = []

print('Multi-run all classifiers')

for i in range(0, 1):
    scaled_X_train, scaled_X_test, flat_y_train, flat_y_test = split_data(all_patient_features, patient_epoch_length, subj_types, 0.3)
    y_test_res.append(flat_y_test)

    # Training random forest model
    y_pred, accuracy = run_rf(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test,)

    # Making confusion matrix
    cm = confusion_matrix(flat_y_test, y_pred, normalize='true')

    rf_res.append(y_pred)
    rf_acc.append(accuracy)
    rf_cm.append(cm)

    print('Random forest: ', accuracy)


    # Training KNN model
    y_pred, accuracy = run_knn(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test,)

    # Making confusion matrix
    cm = confusion_matrix(flat_y_test, y_pred, normalize='true')

    knn_res.append(y_pred)
    knn_acc.append(accuracy)
    knn_cm.append(cm)

    print('KNN: ', accuracy)


    # Training SVM model
    y_pred, accuracy = run_svm(scaled_X_train, scaled_X_test, flat_y_train, flat_y_test,)

    # Making confusion matrix
    cm = confusion_matrix(flat_y_test, y_pred, normalize='true')

    svm_res.append(y_pred)
    svm_cm.append(cm)
    svm_acc.append(accuracy)

    print('SVM: ', accuracy)


## Confusion Matrices

In [ ]:
# Confusion matrices for random forest, SVM, and KNN

print('Generating confusion matrices for random forest, SVM, and KNN')

rf_cm_avg = np.mean(rf_cm, axis=0)
knn_cm_avg = np.mean(knn_cm, axis=0)
svm_cm_avg = np.mean(svm_cm, axis=0)

disp = ConfusionMatrixDisplay(confusion_matrix=rf_cm_avg, display_labels=['A','C','F'])
disp.plot()
disp.ax_.get_images()[0].set_clim(0, 0.7)
plt.title(f'Forest, Accuracy: {np.mean(rf_acc):.3}', fontsize=20, fontweight='bold')
plt.ylabel('True label', fontsize=16)
plt.xlabel('Predicted label', fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout()

#plt.savefig('/content/drive/Shared drives/NeurotechX Shared Drive/rf_cm.png')

disp = ConfusionMatrixDisplay(confusion_matrix=knn_cm_avg, display_labels=['A','C','F'])
disp.plot()
disp.ax_.get_images()[0].set_clim(0, 0.7)
plt.title(f'KNN, Accuracy: {np.mean(knn_acc):.3}', fontsize=20, fontweight='bold')
plt.ylabel('True label', fontsize=16)
plt.xlabel('Predicted label', fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout()

#plt.savefig('/content/drive/Shared drives/NeurotechX Shared Drive/knn_cm.png')

disp = ConfusionMatrixDisplay(confusion_matrix=svm_cm_avg, display_labels=['A','C','F'])
disp.plot()
disp.ax_.get_images()[0].set_clim(0, 0.7)
plt.title(f'SVM, Accuracy: {np.mean(svm_acc):.3}', fontsize=20, fontweight='bold')
plt.ylabel('True label', fontsize=16)
plt.xlabel('Predicted label', fontsize=16)
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout()
#plt.savefig('/content/drive/Shared drives/NeurotechX Shared Drive/svm_cm.png')

In [ ]:
# Generating accuracy bar graphs for random forest, SVM, and KNN

print('Generating accuracy bar graphs for random forest, SVM, and KNN')

models = ['Random Forest', 'KNN', 'SVM']
x_pos = np.arange(len(models))

mean_rf = np.mean(rf_acc)
std_rf = np.std(rf_acc)

mean_knn = np.mean(knn_acc)
std_knn = np.std(knn_acc)

mean_svm = np.mean(svm_acc)
std_svm = np.std(svm_acc)

means = [mean_rf, mean_knn, mean_svm]
stds = [std_rf, std_knn, std_svm]

plt.bar(x_pos, means, yerr=stds, align='center', capsize=10, alpha=0.8, color=['#752D8A', '#D7D9B1', '#84ACCE'])
plt.xticks(x_pos, labels=models)
plt.ylim([0, 0.75])
plt.ylabel('Accuracy (%)')
plt.tight_layout()
#plt.savefig('/content/drive/Shared drives/NeurotechX Shared Drive/barplot.png')